# Atomate2 introduction

Atomate2 is an open-source library providing computational workflows for automating first-principles calculations.

## Supported DFT Codes

- ABINIT
- CP2K
- FHI-AIMS
- JDFTx
- Q-Chem
- VASP

  
## Workflows

Some of the workflows available in atomate2 are:

- electronic band structures
- elastic, dielectric, and piezoelectric tensors
- one-shot electron-phonon interactions
- electronic transport using [AMSET](https://hackingmaterials.lbl.gov/amset/)
- phonons and Grüneisen parameter computations using [phonopy](https://github.com/phonopy/phonopy)
- defect formation energy diagrams
- [Lobster](http://www.cohp.de/) bonding analysis with [LobsterPy](https://github.com/JaGeo/LobsterPy)

## MLIP based workflows

Workflows using MLIPs like MACE, CHGNET, M3GNET, GAP, NEP, SevenNet are available for:

- phonons and Grüneisen parameter computations
- elastic tensors


It is easy to customise and compose any of the above workflows.

# Installation and Setup 

Typically, before using workflows available in atomate2, you will need to create a Python environment with the necessary packages and set some configuration files where you specify details like a path to VASP binary and your database information like collection names, usernames, passwords, etc. You can find this in the [atomate2 documentation](https://materialsproject.github.io/atomate2/user/install.html). If you are interested in specifics, let us know, and we will be happy to provide more information.


This has already been set up for you at this school, so we can directly create workflows and submit them to the HPC cluster. To manage and execute your workflows, you can also use [Fireworks](https://materialsproject.github.io/atomate2/user/fireworks.html). However, we will use [jobflow-remote](https://github.com/Matgenix/jobflow-remote), which was introduced in earlier sessions. 

# Workflows

A Workflow is basically a sequence of tasks where the connectivity, execution order, and dependencies of the different subtasks are predefined using `Flow` or `job` objects. For writing the inputs and parsing (reading) the outputs of calculations, atomate2 mainly relies on [pymatgen](https://github.com/materialsproject/pymatgen) and [ase](https://gitlab.com/ase/ase) packages. Thus it might be beneficial to get familiar with these packages. 

The output of a typical workflow in `atomate2` consists of a summary of the most relevant information for the specific calculation in a JSON serializable dict and is commonly referred to as a `TaskDocument`. The name of this `TaskDocument` and information contained within changes depending on the type of calculation and underlying calculator (e.g.:- DFT code, MLIP ) used in the Workflow.

In this session, we will start from very simple workflows and gradually increase the complexity. 

## Running a  structure relaxation workflow

In [ ]:
from pymatgen.core import Structure
from atomate2.vasp.flows.core import DoubleRelaxMaker
from jobflow_remote import submit_flow

In [ ]:
# construct a rock salt MgO structure
mgo_structure = Structure(
    lattice=[[0, 2.13, 2.13], [2.13, 0, 2.13], [2.13, 2.13, 0]],
    species=["Mg", "O"],
    coords=[[0, 0, 0], [0.5, 0.5, 0.5]],
)

#### Check the customizable parameters for initializing the workflow

In [ ]:
DoubleRelaxMaker?

In [ ]:
DoubleRelaxMaker.make?

In [ ]:
relax_flow = DoubleRelaxMaker().make(structure=mgo_structure) # initialize the flow
relax_flow.name = "Relax Flow" # one can also set a custom name to flow 

In [ ]:
resources={"nodes": 1 , "ntasks": 36, "time": "03:00:00"} # define the resources for the jobs in flow
submit_flow(relax_flow, worker="cecam", resources=resources, exec_config="vasp_6.4.3_cecam") # set the resources for job execution

In [ ]:
! jf runner start # optional

In [ ]:
!jf flow list # overview of flows added to the DB

In [ ]:
!jf job list # Get list of jobs in the DB

If you prefer to monitor jobs via a graphical user interface simply run. This should start a gui server which can be opened in a browser

```bash
jf gui
```

#### Access the relax workflow results from the database and download the optimized structure

For core VASP jobs like the one here (Structure Relaxation), the TaskDocument reside in seperate library [emmet](https://github.com/materialsproject/emmet/tree/main) and is named as `TaskDoc` can details can be found [here
](https://github.com/materialsproject/emmet/blob/947ecc8f63397786957d32f9f98ad1268242b324/emmet-core/emmet/core/tasks.py#L330)

So now, we will access this document which is stored in the database after successful run of the workflow.

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
import matplotlib.pyplot as plt

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
# get output
relax1_doc = jobstore.get_output(uuid=relax_flow.jobs[0].uuid, load=True)

# get python TaskDoc object from serialized output retrieved from DB
task_doc_relax1 = MontyDecoder().process_decoded(relax1_doc)

In [ ]:
# get output
relax2_doc = jobstore.get_output(uuid=relax_flow.jobs[1].uuid, load=True)

# get python TaskDoc object from serialized output retrieved from DB
task_doc_relax2 = MontyDecoder().process_decoded(relax2_doc)

In [ ]:
# Access VASP calculation inputs like incar, kpoints, poscar
task_doc_relax2.input.incar

In [ ]:
# Access final energy
task_doc_relax2.output.energy

In [ ]:
# Access forces
task_doc_relax2.output.forces

In [ ]:
# Access stress
task_doc_relax2.output.stress

In [ ]:
# Access / Download final optimized structure
task_doc_relax2.output.structure.to_file("MgO_opt.cif", fmt="cif")

In [ ]:
# Access directory where the calculation ran
task_doc_relax2.dir_name

In [ ]:
# Collect energies for each optimization step from both relaxation tasks
energy_step = [
    step.e_fr_energy for step in task_doc_relax1.calcs_reversed[0].output.ionic_steps
]
energy_step += [
    step.e_fr_energy for step in task_doc_relax2.calcs_reversed[0].output.ionic_steps
]

# Generate step indices
opt_step = list(range(1, len(energy_step) + 1))

# Compute energy difference relative to the first step
del_energy_step = [e - energy_step[0] for e in energy_step]

# Plot results
plt.plot(opt_step, del_energy_step)
plt.ylabel(r'$\Delta E$')
plt.xlabel("Ionic step")
plt.show()

## Running a Electronic bandstructure workflow

In [ ]:
from pymatgen.core import Structure
from atomate2.vasp.flows.core import BandStructureMaker
from atomate2.vasp.powerups import update_user_incar_settings, update_vasp_custodian_handlers
from jobflow_remote import submit_flow

#### Check the customizable parameters for initializing the workflow 

In [ ]:
BandStructureMaker?

In [ ]:
BandStructureMaker.make?

In [ ]:
# load the downloaded optimized MgO structure
opt_mgo_structure = Structure.from_file("MgO_opt.cif")

In [ ]:
# make a band structure flow and obtain the band structure
bandstructure_flow = BandStructureMaker().make(opt_mgo_structure)
bandstructure_flow.name = "Band structure Flow" # one can also set a custom name to flow

One can update VASP calculation inputs and runtime error handlers using powerups.

Below is an simple example where we update VASP INCAR tags (update_user_incar_settings)  and runtime error handlers (update_vasp_custodian_handlers) for a flow. It is also possible to update for specific subtasks

In [ ]:
bandstructure_flow = update_user_incar_settings(bandstructure_flow, {"NPAR": 4, "ALGO": "FAST"})
bandstructure_flow = update_vasp_custodian_handlers(bandstructure_flow, custom_handlers=())

In [ ]:
resources={"nodes": 1 , "ntasks": 36, "time": "03:00:00"} # define the resources for the jobs in flow
submit_flow(bandstructure_flow, worker="cecam", resources=resources, exec_config="vasp_6.4.3_cecam") # set the resources for job execution

#### Access the bandstructure workflow result from the database and plot the DOS and band structure

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
from pymatgen.electronic_structure.plotter import BSPlotter, DosPlotter

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
bandstructure_flow.jobs[1].uuid

In [ ]:
# get output for line bandstructure job 
line_bs_doc = jobstore.get_output(uuid=bandstructure_flow.jobs[2].uuid, load=True)

# get python TaskDoc object from serialized output retrieved from DB
task_doc_line = MontyDecoder().process_decoded(line_bs_doc)

In [ ]:
bandstructure = task_doc_line.vasp_objects['bandstructure']
bs_plotter = BSPlotter(bandstructure)

In [ ]:
bs_plotter.get_plot(ylim=[-6, 20]);

In [ ]:
# get output for uniform bandstructure job and plot DOS
uniform_bs_doc = jobstore.get_output(bandstructure_flow.jobs[1].uuid, load=True)

# get python TaskDoc object from serialized output retrieved from DB
task_doc_uniform = MontyDecoder().process_decoded(uniform_bs_doc)

In [ ]:
dos = task_doc_uniform.vasp_objects['dos']
dos_plotter = DosPlotter()
dos_plotter.add_dos_dict(dos.get_element_dos())
dos_plotter.get_plot(xlim=[-10,20], ylim=[0, 2.5]);

## Running a bonding analysis (VASP+LOBSTER) workflow  and accessing the results

In [ ]:
from pymatviz.structure_viz import structure_3d_plotly
from atomate2.vasp.flows.mp import MPVaspLobsterMaker
from atomate2.vasp.powerups import update_user_incar_settings
from pymatgen.core import Structure
from jobflow_remote import submit_flow

In [ ]:
# Read AlAgS2 structure
AlAgS2_structure = Structure.from_file("AlAgS2.vasp").get_primitive_structure()

In [ ]:
structure_3d_plotly(AlAgS2_structure)

In [ ]:
lobster_flow = MPVaspLobsterMaker(relax_maker=None).make(structure=AlAgS2_structure)
lobster_flow.name = "VASP LOBSTER flow"

lobster_flow = update_user_incar_settings(lobster_flow, {"NPAR": 6, "ALGO": "FAST"})

In [ ]:
resources={"nodes": 1 , "ntasks": 36, "time": "03:00:00"} # define the resources for the jobs in flow

In [ ]:
submit_flow(lobster_flow, worker="cecam", resources=resources, exec_config="vasp_6.4.3_cecam") # set the resources for job execution

In [ ]:
# get overview of the list of jobs added and its status
! jf job list

#### Access the bonding analysis workflow results from the database and plot the results

For Lobster workflow, the taskdocument is named as [LobsterTaskDocument](https://github.com/materialsproject/atomate2/blob/b8ff9f444bcecaac474f3779acd83ee3b5ad4ae1/src/atomate2/lobster/schemas.py#L640).

So now, we will access this document which is stored in the database after successful run of the workflow.

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
from pymatgen.electronic_structure.cohp import Cohp
from lobsterpy.plotting import PlainCohpPlotter, InteractiveCohpPlotter

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
# get output doc for lobster runs
lobster_doc = jobstore.get_output(lobster_flow.output.uuid, load=True)

# get python LobsterTaskDocuments object from serialized output retrieved from DB
task_docs_lobster = MontyDecoder().process_decoded(lobster_doc)

In [ ]:
for key, cohp in task_docs_lobster["lobster_task_documents"][0].lobsterpy_data_cation_anion.cohp_plot_data.data.items():
    plotter = PlainCohpPlotter()
    plotter.add_cohp(key, cohp)
    plotter.get_plot(sigma=0.05, ylim=[-10,5])

In [ ]:
# access text description
task_docs_lobster["lobster_task_documents"][0].lobsterpy_text_cation_anion

In [ ]:
# access strongest cation-anion bonds 
task_docs_lobster["lobster_task_documents"][0].strongest_bonds_cation_anion

## Running a Phonon workflow with VASP and accessing the results

In [ ]:
from pymatviz.structure_viz import structure_3d_plotly
from atomate2.vasp.flows.phonons import PhononMaker
from atomate2.vasp.powerups import add_metadata_to_flow
from pymatgen.core import Structure
from jobflow_remote import submit_flow

In [ ]:
# Read si structure
si_structure = Structure.from_file("Si.vasp")

In [ ]:
structure_3d_plotly(si_structure)

In [ ]:
# make a Phonon flow to optimise the structure and obtain the Phonon bandstructure
# It is possible to further customize calculation parameters, but for this case we stick to defaults
phonon_flow = PhononMaker(
    min_length=3.0, # here we set small supercell length for faster calculations (ideally you would use 15-20 angstroms supercell)
    use_symmetrized_structure="conventional",
    create_thermal_displacements=False,
    store_force_constants=False,
    prefer_90_degrees=False,
    generate_frequencies_eigenvectors_kwargs={"tstep": 100, "band_structure_eigenvectors": False},
).make(si_structure)

phonon_flow.name = "Phonon flow"

In [ ]:
resources={"nodes": 1 , "ntasks": 36, "time": "03:00:00"} # define the resources for the jobs in flow

In [ ]:
submit_flow(phonon_flow, worker="cecam", resources=resources, exec_config="vasp_6.4.3_cecam") # set the resources for job execution

In [ ]:
# get overview of the list of jobs added and its status
! jf job list

#### Access the phonon workflow results from the database and plot phonon bandstructure

Each of the atomate2 workflows mostly have a associated [Taskdocument](https://materialsproject.github.io/atomate2/user/key_concepts_overview.html) generated for specific type of calculations. The Taskdocument can be understood simply as summary of most important results from the calculation.

For phonon workflow, the taskdocument is named as [PhononBSDOSDoc](https://github.com/materialsproject/atomate2/blob/b8ff9f444bcecaac474f3779acd83ee3b5ad4ae1/src/atomate2/common/schemas/phonons.py#L136). 

So now, we will access this document which is stored in the database after successful run of the workflow.

In [ ]:
from jobflow_remote import get_jobstore
from monty.serialization import MontyDecoder
from pymatgen.phonon.bandstructure import PhononBandStructureSymmLine
from pymatgen.phonon.dos import PhononDos
from pymatgen.phonon.plotter import PhononBSPlotter, PhononDosPlotter

In [ ]:
# connect to the database where the results are stored
jobstore = get_jobstore()
jobstore.connect()

In [ ]:
# get output doc
phonon_doc = jobstore.get_output(phonon_flow.output.uuid, load=True)

# get python PhononBSDOSDoc object from serialized output retrieved from DB
task_doc_phonon = MontyDecoder().process_decoded(phonon_doc)

In [ ]:
# initialize dos plotter and visualize dos plot
dos_plot = PhononDosPlotter()
dos_plot.add_dos(label="Total Dos", dos=task_doc_phonon.phonon_dos)
dos_plot.get_plot();

# initialize Phonon bandstructure plotter and visualize band structure plot
bs_plot = PhononBSPlotter(bs=task_doc_phonon.phonon_bandstructure)
bs_plot.get_plot();

# Optional 

- Run Elastic constant or a Equation of state workflow and access the results